IMPORT NECESSARY LIBRARIES

In [1]:
from langchain.llms import GooglePalm
from langchain.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain
from langchain.prompts import SemanticSimilarityExampleSelector
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.prompts import FewShotPromptTemplate
from langchain.chains.sql_database.prompt import PROMPT_SUFFIX, _mysql_prompt
from langchain.prompts.prompt import PromptTemplate

In [2]:
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.agents.agent_types import AgentType
from langchain.chat_models import ChatOpenAI
from langchain.sql_database import SQLDatabase
from langchain.prompts.chat import ChatPromptTemplate
from sqlalchemy import create_engine

CONNEC TO LLM USING API

In [5]:
from langchain.llms import GooglePalm
api_key=" "
llm = GooglePalm(google_api_key=api_key, temperature=0.2)

CONNECTION TO THE DATABASE

In [7]:
from sqlalchemy import create_engine

#  connection string
connection_string = (
    'mssql+pyodbc:///?odbc_connect='
    'DRIVER={ODBC Driver 18 for SQL Server};'
    'SERVER=surety-dev.database.windows.net,1433;'
    'DATABASE=;'
    'UID=;'
    'PWD=;'
)

db_engine = create_engine(connection_string)


In [8]:

db=SQLDatabase(db_engine)

In [22]:
sql_toolkit=SQLDatabaseToolkit(db=db,llm=llm)


CREATION OF THE PROMPT

In [20]:
prompt=ChatPromptTemplate.from_messages(
    [
        ("system",
        """
        you are a very intelligent AI assitasnt who is expert in identifying relevant questions from user and converting into sql queriesa to generate correcrt answer.
        Please use the below context to write the microsoft sql queries , dont use mysql queries.
       context:
       you must query against the connected database, it has total 64 tables that is ApplicableIndustries, ApplicableJurisdictionDoc, ApplicableJurisdictions, 
       ApplicableProducts, AuthorityMaster, ComplianceAssessment, ComplianceAssessmentAudit, ComplianceCommentary, ControlCategoryLabelMatrix, 
       ControlCategoryMaster, ControlCategoryMatrix, ControlEvidanceMatrix, ControlLabelMaster, ControleEvidenceSegmentType, ControlsData,
         DocCtrlEvdncSegmSubmitStatus, DocWiseWorkFlowStatus, DocumentOwnerMapping, DocumentRequestDetails, DocumentRequestHeader, 
         DocumentRequestRejectReason, DocumentScoreLog, DocumentSegmentData, DocumentSegmentMatrix, DocumentSegmentType, DocumentSubmitStepMaster,
         DocumentSubmitSteps, DocumentTimeline, DocumentUserRoleMapping, EmailNotificationEvents, EmailNotificationLog, EmailNotificationTemplates, 
         Entities, EvidenceData, Geographies, IndustryMaster, JurisdictionMaster, LinkedControlMatrix, LinkedSegmentMatrix, ModuleMaster, 
         NotificationEventRoleMapping, OrgTypeMaster, OrganisationMaster, OrganizationWideReports, OwnerType, ProductMaster, RegulationSubType, 
         RegulationTypeMaster, RegulatoryDocument, RegulatoryDocumentAudit, ReviewAssessment, ReviewAssessmentAudit, ReviewCommentary, RoleMaster, 
         RoleModuleMatrix, ScoreMaster, SegmentControlMatrix, SingleRegulationDocumentReports, StatusMaster, StatusMasterNew, UserMaster, sysdiagrams, 
         tblException, tempcategorylabel
         Below is the given table name and the columns present

         Table Name: LinkedSegmentMatrix
Columns: NestedSegmentId, DocumentSegmentId, LinkedDocumentSegmentId, isActive, WhenEntered, WhenModified

Table Name: EmailNotificationLog
Columns: LogId, EventId, TemplateId, RoleId, ReceiverId, CreatedBy, CreatedOn

Table Name: AuthorityMaster
Columns: AuthorityId, Authority, OrganisationId, isActive, WhenEntered, WhenModified, CreatedBy, ModifiedBy

Table Name: UserMaster
Columns: UserId, Email, ForeName, SurName, PhysicalAddress, JobTitle, UserImage, RoleId, AccessToken, OrganisationId, isActive, WhenEntered, WhenModified

Table Name: DocCtrlEvdncSegmSubmitStatus
Columns: ID, RegDocId, DocCtrlEvdncSegmTypeId, SegmentId, ControlId, IsCreateDelete, Status, IsActive, UpdatedBy, UpdatedOn, CreatedBy, CreatedOn

Table Name: ControlCategoryMaster
Columns: ControlCategoryId, ControlCategory, OrganisationId, isActive, WhenEntered, WhenModified, CreatedBy, ModifiedBy

Table Name: ControlLabelMaster
Columns: ControlLabelId, ControlLabel, OrganisationId, isActive, WhenEntered, WhenModified, CreatedBy, ModifiedBy

Table Name: tblException
Columns: ExceptionID, ExceptionDesc, ExceptionMethodOrEvent, CreatedBy, CreatedOn

Table Name: EvidenceData
Columns: EvidenceId, EvidenceDescription, OrganisationId, isActive, WhenEntered, WhenModified, CreatedBy, ModifiedBy

Table Name: DocumentOwnerMapping
Columns: ID, RegDocId, SegmentId, ControlId, EvidenceId, UserId, OwnerTypeId, IsActive, UpdatedBy, UpdatedOn, CreatedBy, CreatedOn

Table Name: IndustryMaster
Columns: IndustryId, Industry, OrganisationId, isActive, WhenEntered, WhenModified, CreatedBy, ModifiedBy

Table Name: JurisdictionMaster
Columns: JurisdictionId, JurisdictionName, OrganisationId, isActive, WhenEntered, WhenModified, CreatedBy, ModifiedBy, GeographyId

Table Name: ProductMaster
Columns: ProductId, Product, OrganisationId, isActive, WhenEntered, WhenModified, CreatedBy, ModifiedBy

Table Name: RegulationSubType
Columns: RegulationSubTypeId, RegulationSubType, OrganisationId, isActive, WhenEntered, WhenModified, CreatedBy, ModifiedBy, RegulationTypeId

Table Name: ControlsData
Columns: ControlId, ControlName, ControlDescription, RuleLevel, OrganisationId, isActive, WhenEntered, WhenModified, CreatedBy, ModifiedBy, ControlRefId

Table Name: RegulationTypeMaster
Columns: RegulationTypeId, RegulationType, OrganisationId, isActive, WhenEntered, WhenModified, CreatedBy, ModifiedBy

Table Name: RegulatoryDocument
Columns: RegulatoryDocumentId, DocumentTitle, PublicationSource, VersionNumber, PublicationDate, ImplementationDate, TargetComplianceDate, TargetPercentage, NextCertificationDate, AuthorityId, DocumentName, RegulationTypeId, RegulationSubTypeId, StatusId, OrganisationId, isActive, WhenEntered, WhenModified, CreatedBy, ModifiedBy, TotalPages, isReviewerNeeded, ReviewerId, ComplianceOffId, OWNERID

Table Name: ApplicableIndustries
Columns: ApplicableIndustyId, RegulatoryDocumentId, IndustryId, isActive, WhenEntered, WhenModified

Table Name: SegmentControlMatrix
Columns: SegmentControlMatrixId, ControlId, DocumentSegmentId, CreatedBy, isActive, WhenEntered, WhenModified, StatusId

Table Name: DocumentRequestRejectReason
Columns: DocRequestRejReasonId, DocRequestHdrId, ReqSenderReason, CreateOn, CreatedBy

Table Name: ApplicableJurisdictionDoc
Columns: ApplicableJurisdictionID, RegulatoryDocumentId, JurisdictionId, isActive, WhenEntered, WhenModified

Table Name: LinkedControlMatrix
Columns: LinkedControlMatrixId, ControlId, LinkedControlId, isActive, WhenEntered, WhenModified

Table Name: ApplicableJurisdictions
Columns: ApplicableJurisdictionId, ControlId, JurisdictionId, isActive, WhenEntered, WhenModified

Table Name: DocumentTimeline
Columns: DocumentTimelineId, RegulatoryDocumentId, JurisdictionID, StatusId, ActionedBy, Comment, ActivityType, WhenEntered

Table Name: DocumentRequestHeader
Columns: DocRequestId, RegDocId, CtrlEvdncSegmType, CreateDeleteType, isActive, CreateOn, CreatedBy

Table Name: ApplicableProducts
Columns: ApplicableProductId, RegulatoryDocumentId, ProductId, isActive, WhenEntered, WhenModified

Table Name: ComplianceAssessment
Columns: ComplianceAssessmentId, SegmentControlMatrixId, JurisdictionID, EvidenceCollected, ScoreId, Actionedby, isActive, OrganisationId, WhenEntered, WhenModified

Table Name: ScoreMaster
Columns: ScoreId, ScoreWeight, ScoreTitle, isActive, WhenEntered, WhenModified

Table Name: ComplianceCommentary
Columns: ComplianceCommentaryId, ComplianceAssessmentId, Comments, ScoreId, Actionedby, isActive, OrganisationId, WhenEntered, WhenModified

Table Name: ControlCategoryLabelMatrix
Columns: ControlCategoryLabelId, ControlCategoryId, ControlLabelId, isActive, WhenEntered, WhenModified

Table Name: ControlCategoryMatrix
Columns: ControlCategoryId, ControlId, ControlCategoryLabelId, isActive, WhenEntered, WhenModified

Table Name: ControlEvidanceMatrix
Columns: ControlEvidanceMatrixId, ControlId, EvidenceId, isActive, WhenEntered, WhenModified, StatusId

Table Name: DocumentRequestDetails
Columns: DocRequestDetailsId, DocRequestId, ReqSenderUserId, ReqReceiverUserId, isActive, CreatedBy, CreateOn, Status

Table Name: DocumentScoreLog
Columns: DocumentScoreLogId, RegulatoryDocumentId, JurisdictionId, TargetScore, ValidatedScore, NonValidatedScore, WhenEntered

Table Name: RegulatoryDocumentAudit
Columns: RegulatoryDocumentAuditId, RegulatoryDocumentId, StatusId, WhenEntered, ModifiedBy, Action, TargetComplianceDate, TargetPercentage

Table Name: DocumentSegmentMatrix
Columns: DocumentSegmentMatrixId, RegulatoryDocumentId, DocumentSegmentId, CreatedBy, isActive, WhenEntered, WhenModified, StatusId

Table Name: ReviewAssessmentAudit
Columns: ReviewAssessmentAuditId, ReviewAssessmentId, ScoreId, Actionedby, WhenEntered

Table Name: ControleEvidenceSegmentType
Columns: ControleEvidenceSegmentTypeId, CtrlEvdncSegmType, isActive, WhenEntered, WhenModified

Table Name: Entities
Columns: EntityId, Entity, OrganisationId, isActive, WhenEntered, WhenModified, CreatedBy, ModifiedBy

Table Name: ComplianceAssessmentAudit
Columns: ComplianceAssessmentAuditId, ComplianceAssessmentId, ScoreId, WhenEntered, Actionedby

Table Name: OrganisationMaster
Columns: OrganisationId, OrganisationName, OrganisationTypeId, OrgType, Logo, PrimaryColor, isActive, WhenEntered, WhenModified, Description

Table Name: ReviewAssessment
Columns: ReviewAssessmentId, SegmentControlMatrixId, JurisdictionID, ScoreId, Actionedby, isActive, OrganisationId, WhenEntered, WhenModified

Table Name: DocumentSubmitStepMaster
Columns: StepId, StepCode, StepCodeDesc, isActive, WhenEntered, WhenModified

Table Name: StatusMaster
Columns: StatusId, Status, isActive, WhenEntered, WhenModified

Table Name: ReviewCommentary
Columns: ReviewCommentaryId, ReviewAssessmentId, Comments, ScoreId, Actionedby, isActive, OrganisationId, WhenEntered, WhenModified

Table Name: DocumentSubmitSteps
Columns: ID, RegDocId, StepCode, IsActive, UpdatedBy, UpdatedOn, CreatedBy, CreatedOn

Table Name: OrgTypeMaster
Columns: OrganisationTypeId, OrganisationType, isActive, WhenEntered, WhenModified

Table Name: OrganizationWideReports
Columns: OrganizationWideReportId, OrganisationId, StartDate, EndDate, JurisdictionId, IsValidated, ExecutiveSummary, TargetedAction, ReportNo, ReportDocumentName, CreatedBy, WhenEntered

Table Name: DocWiseWorkFlowStatus
Columns: ID, RegDocId, WFStatus, CreatedBy, CreatedOn, ModifiedOn, ModifiedBy

Table Name: DocumentUserRoleMapping
Columns: ID, UserId, RoleId, IsActive, UpdatedBy, UpdatedOn, CreatedBy, CreatedOn

Table Name: SingleRegulationDocumentReports
Columns: SingleRegulationDocumentReportId, RegulatoryDocumentId, JurisdictionId, IsValidated, ExecutiveSummary, TargetedAction, ReportNo, ReportDocumentName, CreatedBy, WhenEntered

Table Name: RoleMaster
Columns: RoleId, Role, isActive, WhenEntered, WhenModifi
 
       
       
       As an expert you must use joins whenever required.
        """
        ),
        ("user","{question}: ")
    ]
)

In [ ]:
# give me the list of regulatory authorities
#How many regulations must we comply in UNITED KINGDOM
#Give me list of analyst
#List out the regulation that implies to India
#ive me a list of controls that are related to risk
#Give the list of segments which contain the word risk

AGENT CREATION 

In [28]:
agent=create_sql_agent(llm=llm,toolkit=sql_toolkit,agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,handle_parsing_errors=True,verbose=True)
agent.run(prompt.format_prompt(question="SELECT JobTitle FROM UserMaster WHERE ForeName = 'Randeep'"))



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: 
Observation: ApplicableIndustries, ApplicableJurisdictionDoc, ApplicableJurisdictions, ApplicableProducts, AuthorityMaster, ComplianceAssessment, ComplianceAssessmentAudit, ComplianceCommentary, ControlCategoryLabelMatrix, ControlCategoryMaster, ControlCategoryMatrix, ControlEvidanceMatrix, ControlLabelMaster, ControleEvidenceSegmentType, ControlsData, DocCtrlEvdncSegmSubmitStatus, DocWiseWorkFlowStatus, DocumentOwnerMapping, DocumentRequestDetails, DocumentRequestHeader, DocumentRequestRejectReason, DocumentScoreLog, DocumentSegmentData, DocumentSegmentMatrix, DocumentSegmentType, DocumentSubmitStepMaster, DocumentSubmitSteps, DocumentTimeline, DocumentUserRoleMapping, EmailNotificationEvents, EmailNotificationLog, EmailNotificationTemplates, Entities, EvidenceData, Geographies, IndustryMaster, JurisdictionMaster, LinkedControlMatrix, LinkedSegmentMatrix, ModuleMaster, NotificationEventRoleMapping, OrgTy

'No user found with ForeName = Randeep'